# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_enunciados.csv', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Using TensorFlow backend.


Found 15387 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [9]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos2.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 8892 = 57.78904269838175


# Treinamento

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0909 12:46:56.653753 140390040348480 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0909 12:46:56.675617 140390040348480 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0909 12:46:56.679681 140390040348480 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0909 12:46:56.687321 140390040348480 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 1,815,546
Trainable params: 276,746
Non-trainable params: 1,538,800
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 54s 5ms/step - loss: 0.9184 - categorical_accuracy: 0.7002 - val_loss: 0.7515 - val_categorical_accuracy: 0.7585
Epoch 2/20
10649/10649 [==============================] - 52s 5ms/step - loss: 0.4089 - categorical_accuracy: 0.8649 - val_loss: 0.5744 - val_categorical_accuracy: 0.8130

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 0.8905 - categorical_accuracy: 0.7136 - val_loss: 0.7135 - val_categorical_accuracy: 0.7751
Epoch 2/20
10649/10649 [==============================] - 56s 5ms/step - loss: 0.3913 - categorical_accuracy: 0.8706 - val_loss: 0.4892 - val_categorical_accuracy: 0.8427
Epoch

In [13]:
from keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               91392     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 1,632,762
Trainable params: 1,632,762
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 24s 2ms/step - loss: 1.3241 - categorical_accuracy: 0.5800 - val_loss: 1.3613 - val_categorical_accuracy: 0.5246
Epoch 2/20
10649/10649 [==============================] - 23s 2ms/step - loss: 0.9851 - categorical_accuracy: 0.6751 - val_loss: 1.4512 - val_categorical_accuracy: 0.5437
Epoch

In [14]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 1,906,938
Trainable params: 1,906,938
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 72s 7ms/step - loss: 0.7879 - categorical_accuracy: 0.7484 - val_loss: 0.7442 - val_categorical_accuracy: 0.7709
Epoch 2/20
10649/10649 [==============================] - 71s 7ms/step - loss: 0.4378 - categorical_accuracy: 0.8583 - val_loss: 0.5688 - val_categorical_accuracy: 0.8179
Epoch

In [15]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(SimpleRNN(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               182784    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 1,726,714
Trainable params: 1,726,714
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 37s 3ms/step - loss: 1.3758 - categorical_accuracy: 0.5643 - val_loss: 1.7626 - val_categorical_accuracy: 0.4424
Epoch 2/20
10649/10649 [==============================] - 36s 3ms/step - loss: 1.0461 - categorical_accuracy: 0.6650 - val_loss: 1.6279 - val_categorical_accuracy: 0.4957
Epoch

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(GRU(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               548352    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 2,092,282
Trainable params: 2,092,282
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 104s 10ms/step - loss: 0.8695 - categorical_accuracy: 0.7221 - val_loss: 0.7247 - val_categorical_accuracy: 0.7702
Epoch 2/20
10649/10649 [==============================] - 101s 10ms/step - loss: 0.4081 - categorical_accuracy: 0.8703 - val_loss: 0.5779 - val_categorical_accuracy: 0.8119
E

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 2,275,066
Trainable params: 2,275,066
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 125s 12ms/step - loss: 0.8014 - categorical_accuracy: 0.7440 - val_loss: 0.7148 - val_categorical_accuracy: 0.7822
Epoch 2/20
10649/10649 [==============================] - 123s 12ms/step - loss: 0.4658 - categorical_accuracy: 0.8503 - val_loss: 0.6544 - val_categorical_accuracy: 0.7927
E

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(128))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 1,628,026
Trainable params: 1,628,026
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 1.0142 - categorical_accuracy: 0.6751 - val_loss: 0.7225 - val_categorical_accuracy: 0.7747
Epoch 2/20
10649/10649 [==============================] - 55s 5ms/step - loss: 0.4772 - categorical_accuracy: 0.8488 - val_loss: 0.6888 - val_categorical_accuracy: 0.7766
Epoch

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(64))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_5 (GRU)                  (None, 64)                31680     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 1,571,130
Trainable params: 1,571,130
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 1.2477 - categorical_accuracy: 0.5920 - val_loss: 1.1262 - val_categorical_accuracy: 0.6485
Epoch 2/20
10649/10649 [==============================] - 55s 5ms/step - loss: 0.6707 - categorical_accuracy: 0.7851 - val_loss: 0.7157 - val_categorical_accuracy: 0.7766
Epoch

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_6 (GRU)                  (None, 512)               941568    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                5130      
Total params: 2,485,498
Trainable params: 2,485,498
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 58s 5ms/step - loss: 0.8581 - categorical_accuracy: 0.7273 - val_loss: 0.6880 - val_categorical_accuracy: 0.7751
Epoch 2/20
10649/10649 [==============================] - 56s 5ms/step - loss: 0.3569 - categorical_accuracy: 0.8834 - val_loss: 0.5155 - val_categorical_accuracy: 0.8348
Epoch

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total params: 1,847,162
Trainable params: 1,847,162
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 0.8890 - categorical_accuracy: 0.7131 - val_loss: 0.7787 - val_categorical_accuracy: 0.7371
Epoch 2/20
10649/10649 [====================

In [22]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_13 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_14 (Dense)             (None, 10)                650       
Total params: 1,830,074
Trainable params: 1,830,074
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 58s 5ms/step - loss: 0.9078 - categorical_accuracy: 0.7063 - val_loss: 0.7732 - val_categorical_accuracy: 0.7668
Epoch 2/20
10649/10649 [====================

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_15 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_16 (Dense)             (None, 10)                330       
Total params: 1,821,530
Trainable params: 1,821,530
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 58s 5ms/step - loss: 0.9006 - categorical_accuracy: 0.7079 - val_loss: 0.7145 - val_categorical_accuracy: 0.7702
Epoch 2/20
10649/10649 [====================

In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0909 14:50:45.826315 140390040348480 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 67s 6ms/step - loss: 1.0337 - categorical_accuracy: 0.6649 - val_loss: 0.8540 - val_categorical_accuracy: 0.7431
Epoch 2/20
10649/10649 [==============================] - 65s 6ms/step - loss: 0.5138 - categorical_accuracy: 0.8342 - val_loss: 0.5878 - val_categorical_accuracy: 0.8074
Epoch

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_18 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 65s 6ms/step - loss: 1.0473 - categorical_accuracy: 0.6597 - val_loss: 0.7797 - val_categorical_accuracy: 0.7537
Epoch 2/20
10649/10649 [==============================] - 63s 6ms/step - loss: 0.5218 - categorical_accuracy: 0.8270 - val_loss: 0.5770 - val_categorical_accuracy: 0.8100
Epoch

In [26]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_12 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 0.9318 - categorical_accuracy: 0.6977 - val_loss: 0.6568 - val_categorical_accuracy: 0.7871
Epoch 2/20
10649/10649 [==============================] - 55s 5ms/step - loss: 0.3694 - categorical_accuracy: 0.8800 - val_loss: 0.5099 - val_categorical_accuracy: 0.8363
Epoch

In [27]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_13 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 58s 5ms/step - loss: 0.9271 - categorical_accuracy: 0.6964 - val_loss: 0.6797 - val_categorical_accuracy: 0.7822
Epoch 2/20
10649/10649 [==============================] - 55s 5ms/step - loss: 0.4100 - categorical_accuracy: 0.8660 - val_loss: 0.6796 - val_categorical_accuracy: 0.7796
Epoch

In [28]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, return_sequences=True))
model.add(GRU(32))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_14 (GRU)                 (None, 200, 256)          274176    
_________________________________________________________________
gru_15 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_21 (Dense)             (None, 10)                330       
Total params: 1,841,050
Trainable params: 1,841,050
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 110s 10ms/step - loss: 0.8542 - categorical_accuracy: 0.7297 - val_loss: 0.6092 - val_categorical_accuracy: 0.8104
Epoch 2/20
10649/10649 [==================

In [29]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_16 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                1290      
Total params: 1,847,162
Trainable params: 1,847,162
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 66s 6ms/step - loss: 1.23

In [30]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_17 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_24 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 30s 3ms/step - loss: 1.0046 - categorical_accuracy: 0.6726 - val_loss: 0.8181 - val_categorical_accuracy: 0.7262
Epoch 2/20
10649/10649 [==============================] - 28s 3ms/step - loss: 0.4746 - categorical_accuracy: 0.8451 - val_loss: 0.7465 - val_categorical_accuracy: 0.7462
Epoch